In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-11 11:51:46--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.07s   

2025-10-11 11:51:46 (1.09 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [47]:
import numpy as np
import pandas as pd

In [48]:
df=pd.read_csv("course_lead_scoring.csv")

In [49]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [50]:
df.columns=df.columns.str.lower().str.replace(' ','_')

In [51]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [52]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [53]:
categorical=list(df.dtypes[df.dtypes=='object'].index)

In [129]:
numerical=['number_of_courses_viewed','annual_income','interaction_count','lead_score']

In [55]:
categorical


['lead_source', 'industry', 'employment_status', 'location']

In [56]:
for c in categorical:
    df[c]=df[c].str.lower().str.replace(' ','_')

In [57]:

numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [58]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [59]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [60]:
df[categorical]=df[categorical].fillna('NA')

In [63]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [62]:
df[numerical]=df[numerical].fillna(0.0)

In [77]:
df.industry.mode()

0    retail
Name: industry, dtype: object

In [78]:
from sklearn.model_selection import train_test_split

In [101]:
df_full_train,df_test=train_test_split(df,test_size=0.2,random_state=42)

In [102]:
df_train,df_val=train_test_split(df_full_train,test_size=0.25,random_state=1)

In [103]:
df_train=df_train.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)

In [104]:
y_train=df_train['converted'].values
y_test=df_test['converted'].values
y_val=df_val['converted'].values

In [105]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [106]:
df_full_train=df_full_train.reset_index(drop=True)

In [112]:
df[['interaction_count','number_of_courses_viewed']].corrwith(df['lead_score']).abs()

interaction_count           0.009888
number_of_courses_viewed    0.004879
dtype: float64

In [113]:
df[['annual_income','number_of_courses_viewed']].corrwith(df['interaction_count']).abs()

annual_income               0.027036
number_of_courses_viewed    0.023565
dtype: float64

In [117]:
from sklearn.metrics import mutual_info_score

In [118]:
def mutual_info_churn_score(series):
    return mutual_info_score(series,y_train)

In [119]:
mi=df_train[categorical].apply(mutual_info_churn_score)

In [123]:
mi.sort_values(ascending=False).round(2)

lead_source          0.03
employment_status    0.02
industry             0.01
location             0.00
dtype: float64

In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [133]:
X_train_num=df_train[numerical].values
scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)

In [135]:
df_train.annual_income

0          0.0
1      68218.0
2      50616.0
3      58172.0
4      62120.0
        ...   
871    55222.0
872    73921.0
873    51633.0
874    48700.0
875        0.0
Name: annual_income, Length: 876, dtype: float64

In [144]:
X_train_num[:1,1]

array([-2.17505804])

In [149]:
ohe=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

In [150]:
X_train_cat = ohe.fit_transform(df_train[categorical].values)

In [152]:
ohe.get_feature_names_out()

array(['x0_NA', 'x0_events', 'x0_organic_search', 'x0_paid_ads',
       'x0_referral', 'x0_social_media', 'x1_NA', 'x1_education',
       'x1_finance', 'x1_healthcare', 'x1_manufacturing', 'x1_other',
       'x1_retail', 'x1_technology', 'x2_NA', 'x2_employed',
       'x2_self_employed', 'x2_student', 'x2_unemployed', 'x3_NA',
       'x3_africa', 'x3_asia', 'x3_australia', 'x3_europe',
       'x3_middle_east', 'x3_north_america', 'x3_south_america'],
      dtype=object)

In [153]:
X_train = np.column_stack([X_train_num, X_train_cat])

In [156]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [157]:
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [159]:
X_val_num = df_val[numerical].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[categorical].values)

X_val = np.column_stack([X_val_num, X_val_cat])

In [163]:
y_pred = model.predict_proba(X_val)[:, 1]


In [164]:
from sklearn.metrics import accuracy_score

In [168]:
accuracy=accuracy_score(y_val, y_pred >= 0.5)

In [170]:
round(accuracy,2)

0.88

In [171]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [174]:
cat_not=['lead_source','employment_status','location']

In [177]:
X_train_num=df_train[numerical].values
X_train_num = scaler.fit_transform(X_train_num)

ohe=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_cat = ohe.fit_transform(df_train[cat_not].values)
X_train = np.column_stack([X_train_num, X_train_cat])

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

X_val_num = df_val[numerical].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[cat_not].values)

X_val = np.column_stack([X_val_num, X_val_cat])

y_pred = model.predict_proba(X_val)[:, 1]

accuracy=accuracy_score(y_val, y_pred >= 0.5)
round(accuracy,2)

0.86

In [178]:
cat_not=['industry','lead_source','location']

In [179]:
X_train_num=df_train[numerical].values
X_train_num = scaler.fit_transform(X_train_num)

ohe=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_cat = ohe.fit_transform(df_train[cat_not].values)
X_train = np.column_stack([X_train_num, X_train_cat])

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

X_val_num = df_val[numerical].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[cat_not].values)

X_val = np.column_stack([X_val_num, X_val_cat])

y_pred = model.predict_proba(X_val)[:, 1]

accuracy=accuracy_score(y_val, y_pred >= 0.5)
round(accuracy,2)

0.84

In [183]:
num_not=['number_of_courses_viewed','annual_income'] 

In [185]:
X_train_num=df_train[num_not].values
X_train_num = scaler.fit_transform(X_train_num)

ohe=OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_cat = ohe.fit_transform(df_train[categorical].values)
X_train = np.column_stack([X_train_num, X_train_cat])

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

X_val_num = df_val[num_not].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[categorical].values)

X_val = np.column_stack([X_val_num, X_val_cat])

y_pred = model.predict_proba(X_val)[:, 1]

accuracy=accuracy_score(y_val, y_pred >= 0.5)
round(accuracy,2)

0.75

In [195]:
C=[0.01, 0.1, 1, 10, 100]
score=[]
for r in C:
    X_train_num=df_train[numerical].values
    X_train_num = scaler.fit_transform(X_train_num)
    
    X_train_cat = ohe.fit_transform(df_train[categorical].values)
    X_train = np.column_stack([X_train_num, X_train_cat])
    
    model = LogisticRegression(solver='liblinear', C=r, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    X_val_num = df_val[numerical].values
    X_val_num = scaler.transform(X_val_num)
    
    X_val_cat = ohe.transform(df_val[categorical].values)
    
    X_val = np.column_stack([X_val_num, X_val_cat])
    
    y_pred = model.predict_proba(X_val)[:, 1]
    
    accuracy=accuracy_score(y_val, y_pred >= 0.5)
    score.append(round(accuracy,3))
    

In [196]:
score

[0.867, 0.877, 0.877, 0.874, 0.874]